Run the beds live sql and then wrangle the data into a simple census

In [4]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from pathlib import Path

Assume that we're running from the notebooks directory and need to pull data from ./src/mock/mock.db

In [64]:
sqlite_db = "../src/mock/mock.db"
assert Path(sqlite_db).is_file()

In [65]:
engine = sa.create_engine(f"sqlite:///{sqlite_db}")

In [66]:
df = pd.read_sql('bedsmock', engine)

In [67]:
df.iloc[0]

modified_at        2022-07-18 21:33:11.120572
location_id                         332136309
department                      UCH T08S ARCU
location_string         10201000174^null^null
ovl_admission                             NaT
ovl_hv_id                                 NaN
open_visits_n                             NaN
cvl_admission                             NaT
cvl_discharge                             NaT
cvl_hv_id                                 NaN
ovl_ghost                               False
occupied                                False
patient_class                            None
mrn                                       nan
encounter                                 NaN
date_of_birth                             NaT
lastname                                 None
firstname                                None
beds_id                                     1
Name: 0, dtype: object

So now you want to return a dataframe with the following characteristics
- per department (one row per department)
- count number of beds
- count number of occupied beds
- count number of empty beds
- last date of closed discharge from that department (i.e. if > 1 week and most beds empty then is the department closed)
- number of side rooms?

In [68]:
temp =  df['location_string'].str.split('^', expand=True).rename(columns={0: 'dept', 1: 'room', 2: 'bed'})
for s in ['dept', 'room', 'bed']:
    df[s] = temp[s]

In [69]:
del(temp)

In [70]:
mask = df['bed'].str.lower().isin(['null', 'wait'])
df = df[~mask]

In [71]:
df.shape

(1207, 22)

In [72]:
groups = df.groupby('department')

In [73]:
groups.get_group('GWB L01 CRITICAL CARE')

,modified_at,location_id,department,location_string,ovl_admission,ovl_hv_id,open_visits_n,cvl_admission,cvl_discharge,cvl_hv_id,...,patient_class,mrn,encounter,date_of_birth,lastname,firstname,beds_id,dept,room,bed
192,2022-07-18 21:33:11.120572,184431051,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR01^SR01-01,NaT,NaN,NaN,2022-07-13 13:33:00,2022-07-17 13:09:00,243907.0,...,None,nan,NaN,NaT,None,None,193,1021800003,GWB L01W SR01,SR01-01
193,2022-07-18 21:33:11.120572,183827636,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR02^SR02-02,2022-07-08 19:54:00,832740.0,1.0,2022-07-06 11:25:00,2022-07-07 13:50:00,972144.0,...,INPATIENT,40918596.0,1.044028e+09,1945-11-30,SPENCER,JIMMY,194,1021800003,GWB L01W SR02,SR02-02
194,2022-07-18 21:33:11.120572,182415664,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR03^SR03-03,2022-07-07 15:28:00,907294.0,1.0,2022-07-05 14:55:00,2022-07-06 14:55:00,193002.0,...,INPATIENT,40894561.0,1.021441e+09,1927-07-06,BOWEN,SEAN,195,1021800003,GWB L01W SR03,SR03-03
195,2022-07-18 21:33:11.120572,182077063,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR04^SR04-04,2022-07-17 10:00:00,260433.0,1.0,2022-07-15 08:33:00,2022-07-17 09:09:00,740052.0,...,INPATIENT,40691002.0,1.081936e+09,1912-04-09,WILLIAMS,MICHAEL,196,1021800003,GWB L01W SR04,SR04-04
196,2022-07-18 21:33:11.120572,187151939,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR05^SR05-05,2022-06-29 20:00:00,341136.0,1.0,2022-06-22 17:51:00,2022-06-24 16:34:00,953413.0,...,INPATIENT,40407245.0,1.051223e+09,1908-11-25,LI,BRANDON,197,1021800003,GWB L01W SR05,SR05-05
197,2022-07-18 21:33:11.120572,185482486,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR06^SR06-06,NaT,NaN,NaN,2022-05-27 17:55:00,2022-06-27 15:09:00,369014.0,...,None,nan,NaN,NaT,None,None,198,1021800003,GWB L01W SR06,SR06-06
198,2022-07-18 21:33:11.120572,327696204,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR07^SR07-07,2022-07-12 02:37:00,363125.0,1.0,2022-06-18 17:30:00,2022-07-10 00:38:00,227424.0,...,INPATIENT,40707035.0,1.091324e+09,1908-11-05,JONES,MISTY,199,1021800003,GWB L01W SR07,SR07-07
199,2022-07-18 21:33:11.120572,187840613,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR08^SR08-08,2022-07-18 18:19:00,339243.0,1.0,2022-07-14 10:42:00,2022-07-14 11:41:00,873332.0,...,INPATIENT,40155753.0,1.024506e+09,1940-11-06,PEARSON,KELLI,200,1021800003,GWB L01W SR08,SR08-08
200,2022-07-18 21:33:11.120572,186862692,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR09^SR09-09,NaT,NaN,NaN,2022-07-18 00:15:00,2022-07-18 18:28:00,144479.0,...,None,nan,NaN,NaT,None,None,201,1021800003,GWB L01W SR09,SR09-09
201,2022-07-18 21:33:11.120572,190347917,GWB L01 CRITICAL CARE,1021800003^GWB L01W SR10^SR10-10,2022-07-18 16:32:00,398664.0,1.0,2022-06-19 20:22:00,2022-07-17 10:00:00,459282.0,...,INPATIENT,40099920.0,1.078171e+09,1961-01-09,HOWARD,KENNETH,202,1021800003,GWB L01W SR10,SR10-10


In [56]:
groups.agg(
    beds = ('location_id', 'count')
)

,beds
department,
,1
GWB L01 CRITICAL CARE,12
GWB L01 ELECTIVE SURG,37
GWB L02 EAST (L02E),19
GWB L02 NORTH (L02N),19
GWB L03 EAST (L03E),19
GWB L03 NORTH (L03N),19
GWB L04 EAST (L04E),17
GWB L04 NORTH (L04N),20
